In [ ]:
import sys
sys.path.append("./..")

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import scipy.stats as stats
import torch
import matplotlib.pyplot as plt
#
from misc.plot_utils import plot_couplings, plot_capsules, plot_mat, plot_mat2

# Simulate Routing Scenarios

In [ ]:
def get_c_perfect_dynamic(n_l, n_h, n_samples, pr=0.0):
    if pr > 0:
        n_samples_p = int(pr * n_samples)
        n_samples = n_samples - n_samples_p
        
    vals = torch.randint(0, n_h, size=(n_samples, n_l))
    C = torch.nn.functional.one_hot(vals).float() * 10
    C = torch.softmax(C + 1e-7, dim=-1).numpy()
    
    if pr > 0:
        C2 = get_c_uniform(n_l, n_h, n_samples_p)
        
        C = np.concatenate([C, C2], axis=0)
    return C

def get_c_uniform(n_l, n_h, n_samples):
    C = torch.softmax(torch.ones(n_samples, n_l, n_h), dim=-1).numpy()
    return C

def get_c_rand(n_l, n_h, n_samples, cs=1, pr=0):
    if pr > 0:
        n_samples_p = int(pr * n_samples)
        n_samples = n_samples - n_samples_p
        
    C = torch.softmax(torch.rand(n_samples, n_l, n_h) * cs, dim=-1).numpy()
    
    if pr > 0:
        C2 = get_c_uniform(n_l, n_h, n_samples_p)
        
        C = np.concatenate([C, C2], axis=0)
    return C

def get_c_rate_strength(n_l, n_h, n_samples, cs=None, cr=None, nc=None, pr=0.0):
    """
        pr: pasive rate
        cs: Coupling Strength
        cr: Coupling Rate
        (nc: number of couplings)
        use either cr or nc
    """
    if pr > 0:
        n_samples_p = int(pr * n_samples)
        n_samples = n_samples - n_samples_p
    
    if nc is None:
        nc = int(cr * n_h)
    #
    assert nc > 0
    #
    CC = [get_c_static(n_l, n_h, n_samples // nc, cs=cs) for _ in range(nc)]
    
    if pr > 0:
        C = get_c_uniform(n_l, n_h, n_samples_p)
        CC.append(C)
    
    C = np.concatenate(CC, axis=0)
    return C

def get_c_static(n_l, n_h, n_samples, cs=10, pr=0):
    if pr > 0:
        n_samples_p = int(pr * n_samples)
        n_samples = n_samples - n_samples_p
    vals = torch.randint(0, n_h, size=(n_l,))
    C = torch.nn.functional.one_hot(vals, num_classes=n_h).float() * cs
    C = torch.softmax(C + 1e-7, dim=-1)
    C = C.unsqueeze(0)
    C = C.repeat(n_samples, 1, 1).numpy()
    
    if pr > 0:
        C2 = get_c_uniform(n_l, n_h, n_samples_p)
        C = np.concatenate([C, C2], axis=0)
    
    return C

def calc_norm_entropy(C):
    Cm = C.mean(axis=0)
    Ce = np.sum(Cm * np.log(Cm) * (1/np.log(Cm.shape[1])), axis=1) * -1
    return Ce

## Mean Max & Adjusted Mean Max

In [ ]:
def mean_max(C):
    """
        Indicating strong couplings for lower layer Capsules
        Only For ACTIVE Capsules interpretable!
        C (b, n_l, n_h) c in C in [0, 1]
    """
    return np.max(C, axis=2).mean(axis=0)

def adjusted_mean_max(C, pr):
    """
        mean_max(C), adjusted for pr (0,1) passive capsules
        pr: either a scalar 
            or a vector of activity weights for each capsule
    """
    n_samples, n_l, n_h = C.shape
    assert type(pr) == float or type(pr) == np.ndarray
    amm = (C.max(axis=2).sum(axis=0) - (pr * n_samples) / n_h) / ((1 - pr) * n_samples)
    return amm

In [ ]:
n_l = 16
n_h = 8
pr = np.array([0.9]*n_l)
n_samples = 1000
C = get_c_perfect_dynamic(n_l, n_h, n_samples, pr=pr[0])
#
C0 = C
C1 = C[:100,:,:]
C2 = C[100:, :, :]

In [ ]:
mean_max(C0)

In [ ]:
mean_max(C1)

In [ ]:
adjusted_mean_max(C0, pr)

## Adjusted Mean and Adjusted Std

In [ ]:
def adjusted_mean_old(C, pr):
    n_samples, n_l, n_h = C.shape
    am = (C.sum(axis=0) - (pr * n_samples) / n_h) / ((1 - pr) * n_samples)
    return am

def adjusted_mean(C, pr):
    n_samples, n_l, n_h = C.shape
    if type(pr) == float:
        pr = np.array([pr]*n_l).reshape(n_l,1)
    else:
        pr = pr.reshape(n_l, 1)
    am = (C.sum(axis=0) - (pr * n_samples) / n_h) / ((1 - pr) * n_samples)
    return am

def adjusted_std_old(C, pr):
    n_samples, n_l, n_h = C.shape
    amc = adjusted_mean_old(C, pr)
    asd = (np.sum((C - amc)**2, axis=0) - (pr * n_samples * (1/n_h - amc)**2)) / ((1 - pr) * n_samples)
    asd = np.sqrt(asd + 1e-5)
    return asd

def adjusted_std(C, pr):
    n_samples, n_l, n_h = C.shape
    if type(pr) == float:
        pr = np.array([pr]*n_l).reshape(n_l,1)
    else:
        pr = pr.reshape(n_l, 1)
    am = adjusted_mean(C, pr)
    asd = (np.sum((C - am)**2, axis=0) - (pr * n_samples * (1/n_h - am)**2)) / ((1 - pr) * n_samples)
    assert asd.min() > -1e-6, "numerics {}".format(asd.min())
    asd = np.maximum(0,asd)
    asd = np.sqrt(asd)
    return asd

In [ ]:
n_l = 16
n_h = 8
pr = np.array([0.9] * n_l)
n_samples = 1000
C = get_c_perfect_dynamic(n_l, n_h, n_samples, pr=pr[0])
#
C0 = C
C1 = C[:100,:,:]
C2 = C[100:, :, :]

In [ ]:
fig, axes = plt.subplots(1, 5, figsize=(24, 8))
plot_mat2(C0.mean(axis=0), ax=axes[0], title="m(C)")
plot_mat2(C1.mean(axis=0), ax=axes[1], title="m(C_active)")
plot_mat2(adjusted_mean_old(C0, pr[0]), ax=axes[2], title="old am(C)")
plot_mat2(adjusted_mean(C0, pr), ax=axes[3], title="am(C)")
plot_mat2(C2.mean(axis=0), ax=axes[4], title="m(C_passive)")

In [ ]:
fig, axes = plt.subplots(1, 5, figsize=(24, 8))
plot_mat2(C0.std(axis=0), ax=axes[0], title="std(C)")
plot_mat2(C1.std(axis=0), ax=axes[1], title="std(C_active)")
plot_mat2(adjusted_std_old(C0, pr[0]), ax=axes[2], title="old_astd(C)")
plot_mat2(adjusted_std(C0, pr), ax=axes[3], title="astd(C)")
plot_mat2(C2.std(axis=0), ax=axes[4], title="std(C_passive)")

# Dynamic Coefficient

In [ ]:
def dynamics2(C):
    n_samples, n_l, n_h = C.shape
    std_pr = np.sqrt(1/n_h * (1 - 1/n_h))
    msd = C.std(axis=0).mean(axis=1)
    mm = mean_max(C)
    dyn = msd / (mm * std_pr)
    return dyn

def dynamics(C):
    n_samples, n_l, n_h = C.shape
    std_pr = np.sqrt(1/n_h * (1 - 1/n_h))
    msd = C.std(axis=0).mean(axis=1)
    mm = C.max(axis=(0,2))
    dyn = msd / (mm * std_pr)
    return dyn

def adjusted_dyn_old(C, pr):
    n_samples, n_l, n_h = C.shape
    std_pr = np.sqrt(1/n_h * (1 - 1/n_h))
    masd = adjusted_std(C, pr).mean(axis=1)
    mm = C.max(axis=(0,2))
    dyn = masd / (mm * std_pr)
    return dyn

def adjusted_dynamics2(C, pr):
    n_samples, n_l, n_h = C.shape
    std_pr = np.sqrt(1/n_h * (1 - 1/n_h))
    masd = adjusted_std(C, pr).mean(axis=1)
    mma = adjusted_mean_max(C, pr)
    dyn = masd / (mma * std_pr)
    return dyn

def adjusted_dyn(C, pr):
    n_samples, n_l, n_h = C.shape
    std_pr = np.sqrt(1/n_h * (1 - 1/n_h))
    masd = adjusted_std(C, pr).mean(axis=1)
    #mma = adjusted_mean_max(C, pr)
    mx = C.max(axis=(0,2)) 
    dyn = masd / (std_pr * mx)
    return dyn

In [ ]:
n_l = 16
n_h = 8
pr = np.array([0.9] * n_l)
n_samples = 10000
C = get_c_perfect_dynamic(n_l, n_h, n_samples, pr=pr[0])
#
C0 = C
C1 = C[:100,:,:]
C2 = C[100:, :, :]
#

In [ ]:
adjusted_dyn(C, pr)

In [ ]:
dynamics(C1)

# Test Metrics

In [ ]:
def print_metrics(C, pr):
    mdy = adjusted_dyn(C, pr).mean()
    mma = adjusted_mean_max(C, pr).mean()
    print("mdy: {:.3f}, mma: {:.3f}".format(mdy, mma))

### Perfect Routing

In [ ]:
n_l = 20
n_h = 10
n_samples = 10000

C = get_c_perfect_dynamic(n_l, n_h, n_samples, pr=0)
#
C_mu = C.mean(axis=0)
C_sd = C.std(axis=0)
C_mx = C.max(axis=0)
#
fig, axes = plt.subplots(1, 3, figsize=(20, 10))
plot_mat2(C_mu, ax=axes[0], vmin=0, vmax=1, title="mean")
plot_mat2(C_sd, ax=axes[1], vmin=0, vmax=0.5, title="sd")
plot_mat2(C_mx, ax=axes[2], vmin=0, vmax=1, title="max")
plt.show()

In [ ]:
for n_l in [20, 40, 60]:
    for n_h in [10, 20, 40]:
        for pr in [0, 0.1, 0.4, 0.8]:
            pr = np.array([pr] * n_l)
            C = get_c_perfect_dynamic(n_l, n_h, n_samples, pr=pr[0])
            print_metrics(C, pr)

### Uniform Routing

In [ ]:
n_l = 20
n_h = 10
n_samples = 10000

C = get_c_uniform(n_l, n_h, n_samples)
C_mu = C.mean(axis=0)
C_sd = C.std(axis=0)
C_mx = C.max(axis=0)
#
fig, axes = plt.subplots(1, 3, figsize=(20, 10))
plot_mat2(C_mu, ax=axes[0], vmin=0, vmax=1, title="mean")
plot_mat2(C_sd, ax=axes[1], vmin=0, vmax=0.5, title="sd")
plot_mat2(C_mx, ax=axes[2], vmin=0, vmax=1, title="max")
plt.show()

In [ ]:
pr = 0
n_samples = 10000
for n_l in [20, 40, 60]:
    for n_h in [10, 20, 40]:
        C = get_c_uniform(n_l, n_h, n_samples)
        print_metrics(C, pr=np.array([pr] * n_l))

### Random Routing

In [ ]:
n_l = 20
n_h = 10
n_samples = 10000

C = get_c_rand(n_l, n_h, n_samples, cs=4)
C_mu = C.mean(axis=0)
C_sd = C.std(axis=0)
C_mx = C.max(axis=0)
#
fig, axes = plt.subplots(1, 3, figsize=(20, 10))
plot_mat2(C_mu, ax=axes[0], vmin=0, vmax=1, title="mean")
plot_mat2(C_sd, ax=axes[1], vmin=0, vmax=0.5, title="sd")
plot_mat2(C_mx, ax=axes[2], vmin=0, vmax=1, title="max")
plt.show()

In [ ]:
for n_l in [20, 40, 60]:
    for n_h in [10, 20, 40]:
        C = get_c_rand(n_l, n_h, n_samples, pr=0, cs=8)
        print_metrics(C, np.array([0] * n_l))

In [ ]:
n_l = 30
n_h = 10
for pr in [0, 0.2, 0.4, 0.8]:
    pr = np.array([pr] * n_l)
    C = get_c_rand(n_l, n_h, n_samples, pr=pr[0], cs=8)
    print_metrics(C, pr)

In [ ]:
n_l = 30
n_h = 10
for pr in [0, 0.2, 0.4, 0.8]:
    pr = np.array([pr] * n_l)
    C = get_c_rand(n_l, n_h, n_samples, pr=pr[0], cs=3)
    print_metrics(C, pr)

### Static Routing

In [ ]:
n_l = 20
n_h = 10
#
n_samples = 10000

C = get_c_static(n_l, n_h, n_samples, cs=4, pr=0.1)
C_mu = C.mean(axis=0)
C_sd = C.std(axis=0)
C_mx = C.max(axis=0)

fig, axes = plt.subplots(1, 3, figsize=(30, 10))
plot_mat2(C_mu, ax=axes[0], vmin=0, vmax=1)
plot_mat2(C_sd, ax=axes[1], vmin=0, vmax=0.5)
plot_mat2(C_mx, ax=axes[2], vmin=0, vmax=1)
plt.show()

In [ ]:
n_l = 20
n_h = 10
for pr in [0, 0.2, 0.4, 0.6]:
    pr = np.array([pr] * n_l)
    C = get_c_static(n_l, n_h, n_samples, cs=4, pr=pr[0])
    print_metrics(C, pr)

In [ ]:
for n_l in [30, 40, 60]:
    for n_h in [10, 20, 30]:
        pr = np.array([0] * n_l)
        C = get_c_static(n_l, n_h, n_samples, cs=4, pr=pr[0])
        print_metrics(C, pr)

In [ ]:
#(0.5 / 0.2)
#masd: vor summe durch max, um relatic skala

In [ ]:
for n_l in [30, 40, 60]:
    for n_h in [10, 20, 30]:
        pr = np.array([0] * n_l)
        C = get_c_static(n_l, n_h, n_samples, cs=8, pr=pr[0])
        print_metrics(C, pr)

In [ ]:
for n_l in [30, 40, 60]:
    for n_h in [10, 20, 30]:
        pr = np.array([0] * n_l)
        C = get_c_static(n_l, n_h, n_samples, cs=2, pr=pr[0])
        print_metrics(C, pr)

### More Dynamic Routing

In [ ]:
n_l = 20
n_h = 10
pr = np.array([0] * n_l)
nc = 2
cs = 10
#
C = get_c_rate_strength(n_l, n_h, n_samples, cs=cs, nc=nc, pr=pr[0])
C_mu = C.mean(axis=0)
C_sd = C.std(axis=0)
C_mx = C.max(axis=0)

fig, axes = plt.subplots(1, 3, figsize=(30, 10))
plot_mat2(C_mu, ax=axes[0], vmin=0, vmax=1)
plot_mat2(C_sd, ax=axes[1], vmin=0, vmax=0.5)
plot_mat2(C_mx, ax=axes[2], vmin=0, vmax=1)
plt.show()

In [ ]:
n_l = 20
n_h = 10
pr = np.array([0] * n_l)
nc = 2
cs = 4

for nc in [2, 4, 6, 8]:
    C = get_c_rate_strength(n_l, n_h, n_samples, cs=cs, nc=nc, pr=pr[0])
    print_metrics(C, pr)
    
n_l = 20
n_h = 10
pr = np.array([0.5] * n_l)
nc = 2
cs = 4

for nc in [2, 4, 6, 8]:
    C = get_c_rate_strength(n_l, n_h, n_samples, cs=cs, nc=nc, pr=pr[0])
    print_metrics(C, pr)

n_l = 20
n_h = 10
pr = np.array([0] * n_l)
nc = 3

for cs in [2, 4, 6, 8, 10]:
    C = get_c_rate_strength(n_l, n_h, n_samples, cs=cs, nc=nc, pr=pr[0])
    print_metrics(C, pr)

n_l = 20
n_h = 10
pr = np.array([0.5] * n_l)
nc = 3

for cs in [2, 4, 6, 8, 10]:
    C = get_c_rate_strength(n_l, n_h, n_samples, cs=cs, nc=nc, pr=pr[0])
    print_metrics(C, pr)

In [ ]:
n_l = 20
n_h = 10
pr = np.array([0] * n_l)
nc = 2
cs = 4

for nc in [2, 4, 6, 8]:
    C = get_c_rate_strength(n_l, n_h, n_samples, cs=cs, nc=nc, pr=pr[0])
    print_metrics(C, pr)

In [ ]:
n_l = 20
n_h = 10
pr = np.array([0.5] * n_l)
nc = 2
cs = 4

for nc in [2, 4, 6, 8]:
    C = get_c_rate_strength(n_l, n_h, n_samples, cs=cs, nc=nc, pr=pr[0])
    print_metrics(C, pr)

In [ ]:
n_l = 20
n_h = 10
pr = np.array([0] * n_l)
nc = 3

for cs in [2, 4, 6, 8, 10]:
    C = get_c_rate_strength(n_l, n_h, n_samples, cs=cs, nc=nc, pr=pr[0])
    print_metrics(C, pr)

In [ ]:
n_l = 20
n_h = 10
pr = np.array([0.5] * n_l)
nc = 3

for cs in [2, 4, 6, 8, 10]:
    C = get_c_rate_strength(n_l, n_h, n_samples, cs=cs, nc=nc, pr=pr[0])
    print_metrics(C, pr)

# Show Data

In [ ]:
import pandas as pd
pd.options.display.float_format = '{:.3f}'.format

In [ ]:
def get_metrics(C, pr):
    mdy = adjusted_dyn(C, pr).mean()
    mma = adjusted_mean_max(C, pr).mean()
    return mdy, mma
    #print("mdy: {:.3f}, mma: {:.3f}".format(mdy, mma))

In [ ]:
n_l = 32
n_h = 16
pr = np.array([0.4] * n_l)
n_samples = 10000

DATA = []
#
C = get_c_perfect_dynamic(n_l, n_h, n_samples, pr=pr[0])
mdy, mma = get_metrics(C, pr)
DATA.append(("Perfect", pr[0], mdy, mma))
#
C = get_c_uniform(n_l, n_h, n_samples)
mdy, mma = get_metrics(C, pr)
DATA.append(("Uniform", pr[0], mdy, mma))
#
C = get_c_rand(n_l, n_h, n_samples, pr=pr[0], cs=10)
mdy, mma = get_metrics(C, pr)
DATA.append(("Random (Strong couplings)", pr[0], mdy, mma))
#
C = get_c_rand(n_l, n_h, n_samples, pr=pr[0], cs=2)
mdy, mma = get_metrics(C, pr)
DATA.append(("Random (Weak couplings)", pr[0], mdy, mma))

C = get_c_static(n_l, n_h, n_samples, cs=10, pr=pr[0])
mdy, mma = get_metrics(C, pr)
DATA.append(("Static (Strong couplings)", pr[0], mdy, mma))

C = get_c_static(n_l, n_h, n_samples, cs=3, pr=pr[0])
mdy, mma = get_metrics(C, pr)
DATA.append(("Static (Weak couplings)", pr[0], mdy, mma))
#
pr = np.array([0.8] * n_l)
#
C = get_c_perfect_dynamic(n_l, n_h, n_samples, pr=pr[0])
mdy, mma = get_metrics(C, pr)
DATA.append(("Perfect", pr[0], mdy, mma))
#
C = get_c_uniform(n_l, n_h, n_samples)
mdy, mma = get_metrics(C, pr)
DATA.append(("Uniform", pr[0], mdy, mma))
#
C = get_c_rand(n_l, n_h, n_samples, pr=pr[0], cs=10)
mdy, mma = get_metrics(C, pr)
DATA.append(("Random (Strong couplings)", pr[0], mdy, mma))
#
C = get_c_rand(n_l, n_h, n_samples, pr=pr[0], cs=2)
mdy, mma = get_metrics(C, pr)
DATA.append(("Random (Weak couplings)", pr[0], mdy, mma))
#
C = get_c_static(n_l, n_h, n_samples, cs=10, pr=pr[0])
mdy, mma = get_metrics(C, pr)
DATA.append(("Static (Strong couplings)", pr[0], mdy, mma))

C = get_c_static(n_l, n_h, n_samples, cs=3, pr=pr[0])
mdy, mma = get_metrics(C, pr)
DATA.append(("Static (Weak couplings)", pr[0], mdy, mma))

In [ ]:
df = pd.DataFrame(DATA, columns=["Routing", "alpha", "dynamics", "mean max"])
df = df.sort_values(by=['Routing'], ascending=True)
df

In [ ]:
n_l = 30
n_h = 15
pr = np.array([0.4] * n_l)
n_samples = 10000

css = [3, 5, 7, 9]
ncs = [1, 4, 7, 10, 13]
#
n_rows, n_cols = len(css), len(ncs)
#
MDY = np.zeros((n_rows, n_cols))
MMA = np.zeros((n_rows, n_cols))
for row_idx  in range(n_rows):
    for col_idx in range(n_cols):
        C = get_c_rate_strength(n_l, n_h, n_samples, cs=css[row_idx], nc=ncs[col_idx], pr=pr[0])
        mdy, mma = get_metrics(C, pr)
        MDY[row_idx][col_idx] = mdy
        MMA[row_idx][col_idx] = mma


In [ ]:
plot_mat2(MDY, vmin=0, vmax=1, row_names=css, col_names=ncs,
          xlabel="Coupling Rate", ylabel="Coupling Strength", title="ROUTING DYNAMIC FACTOR")

In [ ]:
plot_mat2(MMA, vmin=0, vmax=1, row_names=css, col_names=ncs,
          xlabel="Coupling Rate", ylabel="Coupling Strength",
         title="MEAN MAX COUPLINGS")

# Addressing Numerical Stability!

In [ ]:
C = np.array([
    [0.8, 0.1, 0.1],
    [0.9, 0.05, 0.05],
    [0.1, 0.7, 0.2],
    [0.2, 0.7, 0.1],
    [1/3] * 3,
    [1/3] * 3,
    [1/3] * 3,
    [1/3] * 3,
    [1/3] * 3,
    [1/3] * 3,
])
CN = np.maximum(C - 1/3, 0)
CN[CN > 0] += 1/3
C = CN
#
ar = np.mean(np.linalg.norm(C, axis=-1) > 0)
pr =  1 - ar
#

a_tru = np.array([pr])
a_est_b = a_tru + 0.1
a_est_s = a_tru - 0.1


C = C.reshape((-1, 1, 3))
n_samples, n_l, n_h = C.shape
C

In [ ]:
#
# MEAN COUPLINGS
#
EPS = 1e-9
def ma_couplings(C, pr):
    _, n_l, _ = C.shape
    pr = pr.reshape(n_l, 1)
    return C.mean(axis=0) / ( 1 - pr + EPS)

def ma_couplings_o(C, pr):
    n_samples, n_l, n_h = C.shape
    if type(pr) == float:
        pr = np.array([pr]*n_l).reshape(n_l,1)
    else:
        pr = pr.reshape(n_l, 1)
    am = (C.sum(axis=0) - (pr * n_samples) / n_h) / ((1 - pr) * n_samples)
    return am
#
# MEAN MAX
#
def mm_couplings(C):
    return np.max(C, axis=2).mean(axis=0)

def mma_couplings(C, pr):
    _, n_l, _ = C.shape
    assert len(pr) == n_l
    return C.max(axis=-1).mean(axis=0) / (1 - pr.flatten()  + EPS)

def mma_couplings_o(C, pr):
    n_samples, n_l, n_h = C.shape
    assert type(pr) == float or type(pr) == np.ndarray
    amm = (C.max(axis=2).sum(axis=0) - (pr * n_samples) / n_h) / ((1 - pr) * n_samples)
    return amm
#
# STD COUPLINGS
#
def stda_couplings(C, pr):
    ma = ma_couplings(C, pr)
    p0 = (C - ma)**2
    p1 = np.sum(p0, axis=0)
    p2 = (pr * n_samples * ma**2)
    #
    p3 = (p1 - p2)
    p4 = p3 / ((1 - pr) * n_samples)
    p5 = np.sqrt(p4)
    return p5

def stda_couplings(C, pr):
    _, n_l, _ = C.shape
    pr = pr.reshape(n_l, 1)
    ma = ma_couplings(C, pr)
    p1 = ((C - ma)**2).mean(axis=0) / (1 - pr  + EPS)
    p2 = ma**2 * pr / (1 - pr  + EPS)
    return np.sqrt(p1 - p2)

def stda_couplings_o(C, pr):
    n_samples, n_l, n_h = C.shape
    if type(pr) == float:
        pr = np.array([pr]*n_l).reshape(n_l,1)
    else:
        pr = pr.reshape(n_l, 1)
    am = ma_couplings_o(C, pr)
    asd = (np.sum((C - am)**2, axis=0) - (pr * n_samples * (1/n_h - am)**2)) / ((1 - pr) * n_samples)
    #assert asd.min() > -1e-6, "numerics {}".format(asd.min())
    #asd = np.maximum(0,asd)
    asd = np.sqrt(asd)
    return asd

def normalize_couplings(C):
    n_samples, n_l, n_h = C.shape
    CN = np.maximum(C - 1/n_h, 0)
    CN[CN > 0] += 1/n_h
    C = CN
    #
    ar = np.mean(np.linalg.norm(C, axis=-1) > 0, axis=0)
    pr =  1 - ar
    return C, pr

#
# DYNAMICS COEFFICIENT
#
def dyc_capsules(C):
    n_samples, n_l, n_h = C.shape
    std_pr = np.sqrt(1/n_h * (1 - 1/n_h))
    msd = C.std(axis=0).mean(axis=1)
    mm = C.max(axis=(0,2))
    dyn = msd / (mm * std_pr  + EPS)
    return dyn

def dyc_capsules_o(C, pr):
    n_samples, n_l, n_h = C.shape
    std_pr = np.sqrt(1/n_h * (1 - 1/n_h))
    masd = stda_couplings_o(C, pr).mean(axis=1)
    mx = C.max(axis=(0,2)) 
    dyc = masd / (std_pr * mx + EPS)
    return dyc

def dyc_capsules_n(C, pr):
    n_samples, n_l, n_h = C.shape
    std_pr = np.sqrt(1/n_h * (1 - 1/n_h))
    masd = stda_couplings(C, pr).mean(axis=1)
    mx = C.max(axis=(0,2)) 
    dyc = masd / (std_pr * mx + EPS)
    dyc = masd / std_pr
    return dyc

In [ ]:
n_l = 8
n_h = 4
pr_true = np.array([0.4] * n_l)
n_samples = 10000
Ct = get_c_perfect_dynamic(n_l, n_h, n_samples, pr=pr_true[0])
Cn, pr = normalize_couplings(Ct)
#
Cta = Ct[:600]
Ctp = Ct[600:]
#
Cna = Cn[:600]
Cnp = Cn[:600]

### dynamic coefficient

In [ ]:
dyc_capsules(Ct), dyc_capsules(Cn), dyc_capsules(Cta), dyc_capsules(Cna)

In [ ]:
dyc_capsules_n(Cn, pr)

In [ ]:
dyc_capsules_o(Ct, pr), dyc_capsules(Cta)

### adjusted std

In [ ]:
stda_couplings(Cn, pr), Cna.std(axis=0)

In [ ]:
stda_couplings_o(Ct, pr), Cta.std(axis=0)

### adjusted mean max

In [ ]:
mean_max(Ct), mean_max(Cn)

In [ ]:
mma_couplings(Cn, pr), mean_max(Cna)

In [ ]:
mma_couplings_o(Ct, pr), mean_max(Cta)

### adjusted mean

In [ ]:
ma_couplings(Cn, pr), Cna.mean(axis=0)

In [ ]:
ma_couplings_o(Ct, pr), Cta.mean(axis=0)

## test metrics

In [ ]:
def print_metrics(C):
    C, pr = normalize_couplings(C)
    dyc = dyc_capsules_n(C, pr).mean()
    mma = mma_couplings(C, pr).mean()
    print("dyc: {:.3f}, mma: {:.3f}".format(dyc, mma))
    return pr, dyc, mma

### perfect routing

In [ ]:
n_samples = 1000

for n_l in [20, 40, 60]:
    for n_h in [10, 20, 40]:
        for pr in [0, 0.1, 0.4, 0.8]:
            pr = np.array([pr] * n_l)
            C = get_c_perfect_dynamic(n_l, n_h, n_samples, pr=pr[0])
            print_metrics(C)

### uniform routing

In [ ]:
n_samples = 10000
for n_l in [20, 40, 60]:
    for n_h in [10, 20, 40]:
        C = get_c_uniform(n_l, n_h, n_samples)
        print_metrics(C)

### random routing


In [ ]:
n_samples = 10000
n_h = 20
for n_l in [20, 40, 60]:
    C = get_c_rand(n_l, n_h, n_samples, pr=0, cs=8)
    print_metrics(C)

In [ ]:
n_samples = 10000
n_l = 60
for n_h in [20, 40, 60]:
    C = get_c_rand(n_l, n_h, n_samples, pr=0, cs=8)
    print_metrics(C)

In [ ]:
n_l = 30
n_h = 10
for pr in [0, 0.2, 0.4, 0.8]:
    pr = np.array([pr] * n_l)
    C = get_c_rand(n_l, n_h, n_samples, pr=pr[0], cs=8)
    pr_est, dyc, mmn = print_metrics(C)

In [ ]:
n_l = 30
n_h = 10
for pr in [0, 0.2, 0.4, 0.8]:
    pr = np.array([pr] * n_l)
    C = get_c_rand(n_l, n_h, n_samples, pr=pr[0], cs=3)
    pr_est, dyc, mmn = print_metrics(C)

### static routing

In [ ]:
n_l = 30
n_h = 20
for pr in [0, 0.2, 0.4, 0.6]:
    pr = np.array([pr] * n_l)
    C = get_c_static(n_l, n_h, n_samples, cs=2, pr=pr[0])
    pr_est, dyc, mmn = print_metrics(C)

In [ ]:
n_l = 30
n_h = 20
for pr in [0, 0.2, 0.4, 0.6]:
    pr = np.array([pr] * n_l)
    C = get_c_static(n_l, n_h, n_samples, cs=4, pr=pr[0])
    pr_est, dyc, mmn = print_metrics(C)

In [ ]:
n_l = 30
n_h = 20
for pr in [0, 0.2, 0.4, 0.6]:
    pr = np.array([pr] * n_l)
    C = get_c_static(n_l, n_h, n_samples, cs=8, pr=pr[0])
    pr_est, dyc, mmn = print_metrics(C)

In [ ]:
for n_l in [30, 40, 60]:
    for n_h in [10, 20, 30]:
        pr = np.array([0] * n_l)
        C = get_c_static(n_l, n_h, n_samples, cs=8, pr=pr[0])
        pr_est, dyc, mmn = print_metrics(C)

In [ ]:
for n_l in [30, 40, 60]:
    for n_h in [10, 20, 30]:
        pr = np.array([0] * n_l)
        C = get_c_static(n_l, n_h, n_samples, cs=2, pr=pr[0])
        pr_est, dyc, mmn = print_metrics(C)

### dynamic routing

In [ ]:
n_l = 20
n_h = 10
pr = np.array([0] * n_l)
nc = 2
cs = 4

for nc in [2, 4, 6, 8]:
    C = get_c_rate_strength(n_l, n_h, n_samples, cs=cs, nc=nc, pr=pr[0])
    pr_est, dyc, mmn = print_metrics(C)

In [ ]:
n_l = 20
n_h = 10
pr = np.array([0.5] * n_l)
nc = 2
cs = 4

for nc in [2, 4, 6, 8]:
    C = get_c_rate_strength(n_l, n_h, n_samples, cs=cs, nc=nc, pr=pr[0])
    pr_est, dyc, mmn = print_metrics(C)

In [ ]:
n_l = 20
n_h = 10
pr = np.array([0.5] * n_l)
nc = 3

for cs in [1, 2, 3, 4, 5, 6]:
    C = get_c_rate_strength(n_l, n_h, n_samples, cs=cs, nc=nc, pr=pr[0])
    pr_est, dyc, mmn = print_metrics(C)

# Show Data

In [ ]:
import pandas as pd
pd.options.display.float_format = '{:.3f}'.format

In [ ]:
def get_metrics(C, pr):
    C, pr = normalize_couplings(C)
    dyc = dyc_capsules_n(C, pr).mean()
    mma = mma_couplings(C, pr).mean()
    #print("dyc: {:.3f}, mma: {:.3f}".format(dyc, mma))
    return dyc, mma

In [ ]:
n_l = 32
n_h = 16
pr = np.array([0.4] * n_l)
n_samples = 10000

DATA = []
#
C = get_c_perfect_dynamic(n_l, n_h, n_samples, pr=pr[0])
mdy, mma = get_metrics(C, pr)
DATA.append(("Perfect", pr[0], mdy, mma))
#
C = get_c_uniform(n_l, n_h, n_samples)
mdy, mma = get_metrics(C, pr)
DATA.append(("Uniform", pr[0], mdy, mma))
#
C = get_c_rand(n_l, n_h, n_samples, pr=pr[0], cs=10)
mdy, mma = get_metrics(C, pr)
DATA.append(("Random (Strong couplings)", pr[0], mdy, mma))
#
C = get_c_rand(n_l, n_h, n_samples, pr=pr[0], cs=2)
mdy, mma = get_metrics(C, pr)
DATA.append(("Random (Weak couplings)", pr[0], mdy, mma))

C = get_c_static(n_l, n_h, n_samples, cs=10, pr=pr[0])
mdy, mma = get_metrics(C, pr)
DATA.append(("Static (Strong couplings)", pr[0], mdy, mma))

C = get_c_static(n_l, n_h, n_samples, cs=3, pr=pr[0])
mdy, mma = get_metrics(C, pr)
DATA.append(("Static (Weak couplings)", pr[0], mdy, mma))
#
pr = np.array([0.8] * n_l)
#
C = get_c_perfect_dynamic(n_l, n_h, n_samples, pr=pr[0])
mdy, mma = get_metrics(C, pr)
DATA.append(("Perfect", pr[0], mdy, mma))
#
C = get_c_uniform(n_l, n_h, n_samples)
mdy, mma = get_metrics(C, pr)
DATA.append(("Uniform", pr[0], mdy, mma))
#
C = get_c_rand(n_l, n_h, n_samples, pr=pr[0], cs=10)
mdy, mma = get_metrics(C, pr)
DATA.append(("Random (Strong couplings)", pr[0], mdy, mma))
#
C = get_c_rand(n_l, n_h, n_samples, pr=pr[0], cs=2)
mdy, mma = get_metrics(C, pr)
DATA.append(("Random (Weak couplings)", pr[0], mdy, mma))
#
C = get_c_static(n_l, n_h, n_samples, cs=10, pr=pr[0])
mdy, mma = get_metrics(C, pr)
DATA.append(("Static (Strong couplings)", pr[0], mdy, mma))

C = get_c_static(n_l, n_h, n_samples, cs=3, pr=pr[0])
mdy, mma = get_metrics(C, pr)
DATA.append(("Static (Weak couplings)", pr[0], mdy, mma))

In [ ]:
df = pd.DataFrame(DATA, columns=["Routing", "alpha", "DYC", "MMA"])
df = df.sort_values(by=['Routing'], ascending=True)
df

In [ ]:
n_l = 30
n_h = 15
pr = np.array([0.4] * n_l)
n_samples = 10000

css = [3, 5, 7, 9]
ncs = [1, 4, 7, 10, 13]
#
n_rows, n_cols = len(css), len(ncs)
#
MDY = np.zeros((n_rows, n_cols))
MMA = np.zeros((n_rows, n_cols))
for row_idx  in range(n_rows):
    for col_idx in range(n_cols):
        C = get_c_rate_strength(n_l, n_h, n_samples, cs=css[row_idx], nc=ncs[col_idx], pr=pr[0])
        mdy, mma = get_metrics(C, pr)
        MDY[row_idx][col_idx] = mdy
        MMA[row_idx][col_idx] = mma


In [ ]:
plot_mat2(MDY, vmin=0, vmax=1, row_names=css, col_names=ncs,
          xlabel="Coupling Rate", ylabel="Coupling Strength", title="ROUTING DYNAMIC FACTOR")

In [ ]:
plot_mat2(MMA, vmin=0, vmax=1, row_names=css, col_names=ncs,
          xlabel="Coupling Rate", ylabel="Coupling Strength",
         title="MEAN MAX COUPLINGS")

# Version 3 - Addressing Stability & More

In [ ]:
EPS = 1e-9
#
def mma_couplings(C, pr):
    _, n_l, _ = C.shape
    assert len(pr) == n_l
    mma = C.max(axis=-1).mean(axis=0) / (1 - pr.flatten()  + EPS)
    return mma

def lmma_couplings(C, pr):
    mma = mma_couplings(C, pr)
    ws = (1 - pr) / (1 - pr + EPS).sum()
    mma = (ws * mma).sum()
    return mma

def lmm_couplings(C, pr):
    mm = mean_max(C)
    ws = (1 - pr) / (1 - pr + EPS).sum()
    mm = (ws * mm).sum()
    return mm

In [ ]:
EPS = 1e-9
#
def ma_couplings(C, pr):
    _, n_l, _ = C.shape
    pr = pr.reshape(n_l, 1)
    return C.mean(axis=0) / ( 1 - pr + EPS)
#
def stda_couplings(C, pr):
    _, n_l, _ = C.shape
    pr = pr.reshape(n_l, 1)
    ma = ma_couplings(C, pr)
    p1 = ((C - ma)**2).mean(axis=0) / (1 - pr  + EPS)
    p2 = ma**2 * pr / (1 - pr  + EPS)
    p3 = p1 - p2
    p3 = np.maximum(0, p3)
    return np.sqrt(p3)
#
def lmstda_couplings(C, pr):
    mstda = stda_couplings(C, pr).mean(axis=1)
    ws = (1 - pr) / (1 - pr + EPS).sum()
    lmstda = (mstda * ws).sum()
    return lmstda

def dyc_capsules_n1(C, pr):
    n_samples, n_l, n_h = C.shape
    std_pr = np.sqrt(1/n_h * (1 - 1/n_h))
    masd = stda_couplings(C, pr).mean(axis=1)
    dyc = masd / std_pr
    #dyc = dyc.mean() / (1 - pr.mean() + 1e-9)
    return dyc

def dyc_capsules_n2(C, pr):
    n_samples, n_l, n_h = C.shape
    std_pr = np.sqrt(1/n_h * (1 - 1/n_h))
    masd = stda_couplings(C, pr).mean(axis=1)
    mx = C.max(axis=(0,2)) 
    dyc = masd / (std_pr * mx + EPS)
    return dyc

def dyc_capsules_n3(C, pr):
    n_samples, n_l, n_h = C.shape
    std_pr = np.sqrt(1/n_h * (1 - 1/n_h))
    stda = stda_couplings(C, pr)
    mstda = stda.mean(axis=1)
    
def dycm_capsules(C, pr):
    mstda = stda_couplings(C, pr).mean(axis=1)
    mma = mma_couplings(C, pr)
    assert np.all(mstda <= mma)
    dycm = mstda / (mma + EPS)
    return dycm

def ldycm_capsules(C, pr):
    dyc = dycm_capsules(C, pr)
    ws = (1 - pr) / (1 - pr + EPS).sum()
    ldyc = (ws * dyc).sum()
    return ldyc


def dycmx_capsules(C, pr):
    mstda = stda_couplings(C, pr).mean(axis=1)
    mx = C.max(axis=(0,-1))
    assert np.all(mstda <= mx)
    dycmx = mstda / (mx + EPS)
    return dycmx

def ldycmx_capsules(C, pr):
    dycmx = dycmx_capsules(C, pr)
    ws = (1 - pr) / (1 - pr + EPS).sum()
    ldycmx = (ws * dycmx).sum()
    return ldycmx

def dycpr_capsules(C, pr):
    n_samples, n_l, n_h = C.shape
    std_pr = np.sqrt(1/n_h * (1 - 1/n_h))
    mstda = stda_couplings(C, pr).mean(axis=1)
    dycpr = mstda / std_pr
    return dycpr

def ldycpr_capsules(C, pr):
    dycpr = dycpr_capsules(C, pr)
    ws = (1 - pr) / (1 - pr + EPS).sum()
    ldycpr = (ws * dycpr).sum()
    return ldycpr

def dycmpr_capsules(C, pr):
    n_samples, n_l, n_h = C.shape
    mstda = stda_couplings(C, pr).mean(axis=1)
    mma = mma_couplings(C, pr)
    std_pr = np.sqrt(1/n_h * (1 - 1/n_h))
    assert np.all(mstda <= mma)
    dycmpr = mstda / (mma * std_pr + EPS)
    return dycmpr

def ldycmpr_capsules(C, pr):
    dycmpr = dycmpr_capsules(C, pr)
    ws = (1 - pr) / (1 - pr + EPS).sum()
    ldycmpr =  (ws * dycmpr).sum()
    return ldycmpr

def dycmxpr_couplings(C, pr):
    n_samples, n_l, n_h = C.shape
    mstda = stda_couplings(C, pr).mean(axis=1)
    cmx = C.max(axis=(0,-1))
    std_pr = np.sqrt(1/n_h * (1 - 1/n_h))
    assert np.all(mstda <= cmx)
    dycmxpr = mstda / (cmx * std_pr + EPS)
    return dycmxpr

def ldycmxpr_couplings(C, pr):
    n_samples, n_l, n_h = C.shape
    mstda = stda_couplings(C, pr).mean(axis=1)
    cmx = C.max(axis=(0,-1))
    std_pr = np.sqrt(1/n_h * (1 - 1/n_h))
    assert np.all(mstda <= cmx)
    dycmxpr = mstda / (cmx + EPS)
    #
    ws = (1 - pr) / (1 - pr + EPS).sum()
    ldycmxpr =  (ws * dycmxpr).sum() / std_pr
    return ldycmxpr

## test metrics

In [ ]:
def print_metrics(C):
    C, pr = normalize_couplings(C)
    dyc = ldycmxpr_couplings(C, pr)
    mma = lmma_couplings(C, pr)
    print("dyc: {:.3f}, mma: {:.3f}".format(dyc, mma))
    return pr, dyc, mma

### perfect routing

In [ ]:
n_samples = 1000

for n_l in [20, 40, 60]:
    for n_h in [10, 20, 40]:
        for pr in [0, 0.1, 0.4, 0.8]:
            pr = np.array([pr] * n_l)
            C = get_c_perfect_dynamic(n_l, n_h, n_samples, pr=pr[0])
            print_metrics(C)

### uniform routing

In [ ]:
n_samples = 10000
for n_l in [20, 40, 60]:
    for n_h in [10, 20, 40]:
        C = get_c_uniform(n_l, n_h, n_samples)
        print_metrics(C)

### random routing


In [ ]:
n_samples = 10000
n_h = 20
for n_l in [20, 40, 60]:
    C = get_c_rand(n_l, n_h, n_samples, pr=0, cs=8)
    print_metrics(C)

In [ ]:
n_samples = 10000
n_l = 60
for n_h in [20, 40, 60]:
    C = get_c_rand(n_l, n_h, n_samples, pr=0, cs=8)
    print_metrics(C)

In [ ]:
n_l = 30
n_h = 10
for pr in [0, 0.2, 0.4, 0.8]:
    pr = np.array([pr] * n_l)
    C = get_c_rand(n_l, n_h, n_samples, pr=pr[0], cs=2)
    pr_est, dyc, mmn = print_metrics(C)

In [ ]:
n_l = 30
n_h = 10
for pr in [0, 0.2, 0.4, 0.8]:
    pr = np.array([pr] * n_l)
    C = get_c_rand(n_l, n_h, n_samples, pr=pr[0], cs=6)
    pr_est, dyc, mmn = print_metrics(C)

In [ ]:
n_l = 30
n_h = 10
for pr in [0, 0.2, 0.4, 0.8]:
    pr = np.array([pr] * n_l)
    C = get_c_rand(n_l, n_h, n_samples, pr=pr[0], cs=10)
    pr_est, dyc, mmn = print_metrics(C)

### static routing

In [ ]:
n_l = 30
n_h = 20
for pr in [0, 0.2, 0.4, 0.6]:
    pr = np.array([pr] * n_l)
    C = get_c_static(n_l, n_h, n_samples, cs=2, pr=pr[0])
    pr_est, dyc, mmn = print_metrics(C)

In [ ]:
n_l = 30
n_h = 20
for pr in [0, 0.2, 0.4, 0.6]:
    pr = np.array([pr] * n_l)
    C = get_c_static(n_l, n_h, n_samples, cs=4, pr=pr[0])
    pr_est, dyc, mmn = print_metrics(C)

In [ ]:
n_l = 30
n_h = 20
for pr in [0, 0.2, 0.4, 0.6]:
    pr = np.array([pr] * n_l)
    C = get_c_static(n_l, n_h, n_samples, cs=10, pr=pr[0])
    pr_est, dyc, mmn = print_metrics(C)


In [ ]:
for n_l in [30, 40, 60]:
    for n_h in [10, 20, 30]:
        pr = np.array([0] * n_l)
        C = get_c_static(n_l, n_h, n_samples, cs=8, pr=pr[0])
        pr_est, dyc, mmn = print_metrics(C)

In [ ]:
for n_l in [30, 40, 60]:
    for n_h in [10, 20, 30]:
        pr = np.array([0] * n_l)
        C = get_c_static(n_l, n_h, n_samples, cs=2, pr=pr[0])
        pr_est, dyc, mmn = print_metrics(C)

### dynamic routing

In [ ]:
n_l = 20
n_h = 10
pr = np.array([0] * n_l)
cs = 4
n_samples = 10000

for nc in [2, 4, 6, 8, 10]:
    C = get_c_rate_strength(n_l, n_h, n_samples, cs=cs, nc=nc, pr=pr[0])
    pr_est, dyc, mmn = print_metrics(C)

In [ ]:
n_l = 32
n_h = 16
pr = np.array([0.5] * n_l)
cs = 4
n_samples = 10000

for nc in [2, 4, 8, 12, 16]:
    C = get_c_rate_strength(n_l, n_h, n_samples, cs=cs, nc=nc, pr=pr[0])
    pr_est, dyc, mmn = print_metrics(C)

In [ ]:
from misc.metrics import *

In [ ]:
n_l = 32
n_h = 16
pr = np.array([0.5] * n_l)
cs = 4
n_samples = 10000

for nc in [2, 4, 8, 12, 16]:
    C = get_c_rate_strength(n_l, n_h, n_samples, cs=cs, nc=nc, pr=pr[0])

    Cn, pr = normalize_couplings(C, eps_rate=0.5) # this should not have an influence
    
    dycm = dycm_capsules(Cn, pr)
    ldycm = ldycm_capsules(Cn, pr)
    dycmx = dycmx_capsules(Cn, pr)
    ldycmx = ldycmx_capsules(Cn, pr)
    dycpr = dycpr_capsules(Cn, pr)
    ldycpr = ldycpr_capsules(Cn, pr)
    dycmpr = dycmpr_capsules(Cn, pr)
    ldycmpr = ldycmpr_capsules(Cn, pr)
    dycmxpr = dycmxpr_couplings(Cn, pr)
    ldycmxpr = ldycmxpr_couplings(Cn, pr)
    #
    print("ldycm {:.3f} ldycmx {:.3f} ldycpr: {:.3f} ldycmpr {:.3f} ldycmxpr: {:.3f}".format(
        ldycm, ldycmx, ldycpr, ldycmpr, ldycmxpr))

In [ ]:
dycmxpr.mean()

# Show Data

In [ ]:
import pandas as pd
pd.options.display.float_format = '{:.3f}'.format

In [ ]:
def get_metrics(C, pr):
    C, pr = normalize_couplings(C)
    dyc = ldycmxpr_couplings(C, pr)
    #dyc = ldycmpr_capsules(C, pr)
    mma = lmma_couplings(C, pr)
    #print("dyc: {:.3f}, mma: {:.3f}".format(dyc, mma))
    return dyc, mma

In [ ]:
n_l = 32
n_h = 16
pr = np.array([0.4] * n_l)
n_samples = 10000

DATA = []
#
C = get_c_perfect_dynamic(n_l, n_h, n_samples, pr=pr[0])
mdy, mma = get_metrics(C, pr)
DATA.append(("Perfect", pr[0], mdy, mma))
#
C = get_c_uniform(n_l, n_h, n_samples)
mdy, mma = get_metrics(C, pr)
DATA.append(("Uniform", pr[0], mdy, mma))
#
C = get_c_rand(n_l, n_h, n_samples, pr=pr[0], cs=10)
mdy, mma = get_metrics(C, pr)
DATA.append(("Random (Strong couplings)", pr[0], mdy, mma))
#
C = get_c_rand(n_l, n_h, n_samples, pr=pr[0], cs=2)
mdy, mma = get_metrics(C, pr)
DATA.append(("Random (Weak couplings)", pr[0], mdy, mma))

C = get_c_static(n_l, n_h, n_samples, cs=10, pr=pr[0])
mdy, mma = get_metrics(C, pr)
DATA.append(("Static (Strong couplings)", pr[0], mdy, mma))

C = get_c_static(n_l, n_h, n_samples, cs=3, pr=pr[0])
mdy, mma = get_metrics(C, pr)
DATA.append(("Static (Weak couplings)", pr[0], mdy, mma))
#
pr = np.array([0.8] * n_l)
#
C = get_c_perfect_dynamic(n_l, n_h, n_samples, pr=pr[0])
mdy, mma = get_metrics(C, pr)
DATA.append(("Perfect", pr[0], mdy, mma))
#
C = get_c_uniform(n_l, n_h, n_samples)
mdy, mma = get_metrics(C, pr)
DATA.append(("Uniform", pr[0], mdy, mma))
#
C = get_c_rand(n_l, n_h, n_samples, pr=pr[0], cs=10)
mdy, mma = get_metrics(C, pr)
DATA.append(("Random (Strong couplings)", pr[0], mdy, mma))
#
C = get_c_rand(n_l, n_h, n_samples, pr=pr[0], cs=2)
mdy, mma = get_metrics(C, pr)
DATA.append(("Random (Weak couplings)", pr[0], mdy, mma))
#
C = get_c_static(n_l, n_h, n_samples, cs=10, pr=pr[0])
mdy, mma = get_metrics(C, pr)
DATA.append(("Static (Strong couplings)", pr[0], mdy, mma))

C = get_c_static(n_l, n_h, n_samples, cs=3, pr=pr[0])
mdy, mma = get_metrics(C, pr)
DATA.append(("Static (Weak couplings)", pr[0], mdy, mma))

In [ ]:
df = pd.DataFrame(DATA, columns=["Routing", "alpha", "DYC", "MMA"])
df = df.sort_values(by=['Routing'], ascending=True)
df

In [ ]:
df = pd.DataFrame(DATA, columns=["Routing", "alpha", "DYC", "MMA"])
df = df.sort_values(by=['Routing'], ascending=True)
df

In [ ]:
n_l = 30
n_h = 15
pr = np.array([0.4] * n_l)
n_samples = 10000

css = [3, 5, 7, 9]
ncs = [1, 4, 7, 10, 13]
#
n_rows, n_cols = len(css), len(ncs)
#
MDY = np.zeros((n_rows, n_cols))
MMA = np.zeros((n_rows, n_cols))
for row_idx  in range(n_rows):
    for col_idx in range(n_cols):
        C = get_c_rate_strength(n_l, n_h, n_samples, cs=css[row_idx], nc=ncs[col_idx], pr=pr[0])
        mdy, mma = get_metrics(C, pr)
        MDY[row_idx][col_idx] = mdy
        MMA[row_idx][col_idx] = mma


In [ ]:
plot_mat2(MDY, vmin=0, vmax=1, row_names=css, col_names=ncs,
          xlabel="Coupling Rate", ylabel="Coupling Strength", title="ROUTING DYNAMIC FACTOR")

In [ ]:
plot_mat2(MMA, vmin=0, vmax=1, row_names=css, col_names=ncs,
          xlabel="Coupling Rate", ylabel="Coupling Strength",
         title="MEAN MAX COUPLINGS")

# Metrics for different activity rates

In [ ]:
from misc.metrics import *

In [ ]:
def get_c_perfect_dynamic_var(n_l, n_h, n_samples, pr):
    vals = torch.randint(0, n_h, size=(n_samples, n_l))
    C = torch.nn.functional.one_hot(vals).float() * 10
    #
    for idx in range(n_l):
        pri = pr[idx]
        n_pr = int(pri * n_samples)
        idcs = np.random.choice(range(n_samples), n_pr, replace=False)
        C[idcs, idx, :] = 0
    C = torch.softmax(C + 1e-7, dim=-1).numpy()
    return C

def get_c_rate_strength_var2(n_l, n_h, n_samples, cs=None, cr=None, nc=None, pr=None):
    """
        pr: pasive rate
        cs: Coupling Strength
        cr: Coupling Rate
        (nc: number of couplings)
        use either cr or nc
    """
    if nc is None:
        nc = int(cr * n_h)
    #
    assert nc > 0
    #
    CC = [get_c_static(n_l, n_h, n_samples // nc, cs=cs) for _ in range(nc)]
    C = np.concatenate(CC, axis=0)
    for idx in range(n_l):
        pri = pr[idx]
        n_pr = int(pri * C.shape[0])
        idcs = np.random.choice(range(C.shape[0]), n_pr, replace=False)
        C[idcs, idx, :] = 1/n_h
    return C

def get_c_static(n_l, n_h, n_samples, cs=10, pr=0):
    if pr > 0:
        n_samples_p = int(pr * n_samples)
        n_samples = n_samples - n_samples_p
    vals = torch.randint(0, n_h, size=(n_l,))
    C = torch.nn.functional.one_hot(vals, num_classes=n_h).float() * cs
    C = torch.softmax(C + 1e-7, dim=-1)
    C = C.unsqueeze(0)
    C = C.repeat(n_samples, 1, 1).numpy()
    
    if pr > 0:
        C2 = get_c_uniform(n_l, n_h, n_samples_p)
        C = np.concatenate([C, C2], axis=0)
    
    return C

def get_c_rate_strength_var(n_l, n_h, n_samples, pr, cs=10, cr=None, nc=None):
    if nc is None:
        nc = int(cr * n_h)
    n_samples_c = n_samples // nc
    vals = torch.randint(0, n_h, size=(n_l,))
    LH = []
    for idx_l in range(n_l):
        g = np.random.choice(n_h, nc, replace=False)
        LH.append(g)
    LH = np.array(LH)
    C = []
    for idx in range(LH.shape[1]):
        vals = LH[:,idx]
        vals = torch.nn.functional.one_hot(torch.LongTensor(vals), num_classes=n_h) * cs
        vals = torch.softmax(vals + 1e-3, dim=-1)
        vals = vals.unsqueeze(0)
        vals = vals.repeat(n_samples_c, 1, 1).numpy()
        C.append(vals)
    C = np.concatenate(C)
    for idx in range(n_l):
        pri = pr[idx]
        n_pr = int(pri * C.shape[0])
        idcs = np.random.choice(range(C.shape[0]), n_pr, replace=False)
        C[idcs, idx, :] = 1/n_h
    return C

In [ ]:
def get_vibrance(U, C):
    return rates_activities(U, C)


def get_bonding(C):
    Cn, pr = normalize_couplings(C, eps_rate=0.5)
    #
    # mma = mma_couplings(Cn, pr)
    # mm = mm_couplings(C)
    #
    lmma = lmma_couplings(Cn, pr)
    lmm = lmm_couplings(C, pr)
    return lmma, lmm


def get_dynamics(C):
    C, pr = normalize_couplings(C, eps_rate=0.5)
    #
    ldycpr = ldycpr_capsules(C, pr)
    # ldycmpr = ldycmpr_capsules(C, pr)
    ldycmxpr = ldycmxpr_couplings(C, pr)
    return ldycpr, ldycmxpr


In [ ]:
n_samples = 1000

print("{:<7}{:<7}{:<7}{:<7}{:<7}{:<7}".format("pr_tru", "pr_est",
                                              "lmma", "lmm",
                                              "ldycpr", "ldycmxpr"))
for n_l in [20, 40, 60]:
    for n_h in [10, 20, 30]:
        for pr in [0, 0.1, 0.4, 0.8]:
            pr = np.random.rand(n_l)
            C = get_c_perfect_dynamic_var(n_l, n_h, n_samples, pr)
            pr_est = rate_inactive_capsules(C)
            lmma, lmm = get_bonding(C)
            ldycpr, ldycmxpr = get_dynamics(C)
            print("{:.3f}  {:.3f}  {:.3f}  {:.3f}  {:.3f}  {:.3f}".format(pr.mean(), pr_est.mean(), lmma, lmm, ldycpr, ldycmxpr))

In [ ]:
n_samples = 10000

print("{:<7}{:<7}{:<7}{:<7}{:<7}{:<7}  {:<7}".format("pr_tru", "pr_est",
                                              "lmma", "lmm",
                                              "ldycpr", "ldycmxpr", "cmax"))
cs = 10
cr = 0.5
for n_l in [40, 60]:
    for n_h in [10, 20, 30]:
        pr = np.random.rand(n_l)
        C = get_c_rate_strength_var(n_l, n_h, n_samples, pr, cs, cr)
        pr_est = rate_inactive_capsules(C)
        lmma, lmm = get_bonding(C)
        ldycpr, ldycmxpr = get_dynamics(C)
        cmax = C.max(axis=(0,2)).mean()
        print("{:.3f}  {:.3f}  {:.3f}  {:.3f}  {:.3f}  {:.3f}    {:.3f}".format(pr.mean(), pr_est.mean(), lmma, lmm, ldycpr, ldycmxpr, cmax))

In [ ]:
n_samples = 10000

print("{:<7}{:<7}{:<7}{:<7}{:<7}{:<7}  {:<7}".format("pr_tru", "pr_est",
                                              "lmma", "lmm",
                                              "ldycpr", "ldycmxpr", "ldycmpr"))
cs = 10
cr = 0.2

for n_l in [30]:
    for n_h in [30, 40, 50, 60]:
        pr = np.random.rand(n_l)
        C = get_c_rate_strength_var(n_l, n_h, n_samples, pr, cs, cr)
        pr_est = rate_inactive_capsules(C)
        lmma, lmm = get_bonding(C)
        ldycpr, ldycmxpr = get_dynamics(C)
        ldycmpr = ldycmpr_capsules(C, pr_est)
        cmax = C.max(axis=(0,2)).mean()
        print("{:.3f}  {:.3f}  {:.3f}  {:.3f}  {:.3f}  {:.3f}    {:.3f}".format(pr.mean(), pr_est.mean(), lmma, lmm, ldycpr, ldycmxpr, ldycmpr))